# Premade Estimators

This document introduces the TensorFlow programming environment and shows you how to solve the Iris classification problem in TensorFlow.

## The programming stack
As the following illustration shows, TensorFlow provides a programming stack consisting of multiple API layers:

![](https://tensorflow.google.cn/images/tensorflow_programming_environment.png)

We strongly recommend writing TensorFlow programs with the following APIs:

- [Estimators](https://tensorflow.google.cn/programmers_guide/estimators), which represent a complete model. The Estimator API provides methods to train the model, to judge the model's accuracy, and to generate predictions.
- [Datasets](https://tensorflow.google.cn/get_started/datasets_quickstart), which build a data input pipeline. The Dataset API has methods to load and manipulate data, and feed it into your model. The Dataset API meshes well with the Estimators API.


## Overview of programming with Estimators
An Estimator is TensorFlow's high-level representation of a complete model. It handles the details of initialization, logging, saving and restoring, and many other features so you can concentrate on your model. For more details see [Estimators](https://tensorflow.google.cn/programmers_guide/estimators).

An Estimator is any class derived from [tf.estimator.Estimator](https://tensorflow.google.cn/api_docs/python/tf/estimator/Estimator). TensorFlow provides a collection of [pre-made Estimators](https://developers.google.cn/machine-learning/glossary/#pre-made_Estimator) (for example, `LinearRegressor`) to implement common ML algorithms. Beyond those, you may write your own [custom Estimators](https://developers.google.cn/machine-learning/glossary/#custom_Estimator). We recommend using pre-made Estimators when just getting started with TensorFlow. After gaining expertise with the pre-made Estimators, we recommend optimizing your model by creating your own custom Estimators.

To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

- Create one or more input functions.
- Define the model's feature columns.
- Instantiate an Estimator, specifying the feature columns and various hyperparameters.
- Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data.

Let's see how those tasks are implemented for Iris classification.

## Create input functions
You must create input functions to supply data for training, evaluating, and prediction.

An **input function** is a function that returns a [tf.data.Dataset](https://tensorflow.google.cn/api_docs/python/tf/data/Dataset) object which outputs the following two-element tuple:

- [features](https://developers.google.cn/machine-learning/glossary/#feature) - A Python dictionary in which:
    - Each key is the name of a feature.
    - Each value is an array containing all of that feature's values.

- `label` - An array containing the values of the [label](https://developers.google.cn/machine-learning/glossary/#label) for every example.

Just to demonstrate the format of the input function, here's a simple implementation:

In [1]:
def input_evaluation_set():
    features = {'SepalLength': np.array([6.4, 5.0]),
                'SepalWidth':  np.array([2.8, 2.3]),
                'PetalLength': np.array([5.6, 3.3]),
                'PetalWidth':  np.array([2.2, 1.0])}
    labels = np.array([2, 1])
    return features, labels

Your input function may generate the features dictionary and label list any way you like. However, we recommend using TensorFlow's Dataset API, which can parse all sorts of data. At a high level, the Dataset API consists of the following classes:

![](https://tensorflow.google.cn/images/dataset_classes.png)

Where the individual members are:

- Dataset - Base class containing methods to create and transform datasets. Also allows you to initialize a dataset from data in memory, or from a Python generator.
- TextLineDataset - Reads lines from text files.
- TFRecordDataset - Reads records from TFRecord files.
- FixedLengthRecordDataset - Reads fixed size records from binary files.
- Iterator - Provides a way to access one data set element at a time.

The Dataset API can handle a lot of common cases for you. For example, using the Dataset API, you can easily read in records from a large collection of files in parallel and join them into a single stream.

To keep things simple in this example we are going to load the data with [pandas](https://pandas.pydata.org/), and build our input pipeline from this in-memory data.

Here is the input function used for training in this program, which is available in [iris_data.py](https://github.com/tensorflow/models/blob/master/samples/core/get_started/iris_data.py):

In [2]:
def train_input_fc(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle, repeat, and batch the examples.
    return dataset.shuffle(1000).repeat().batch(batch_size)

## Define the feature columns

A [feature column](https://developers.google.cn/machine-learning/glossary/#feature_columns) is an object describing how the model should use raw input data from the features dictionary. When you build an Estimator model, you pass it a list of feature columns that describes each of the features you want the model to use. The [tf.feature_column](https://tensorflow.google.cn/api_docs/python/tf/feature_column) module provides many options for representing data to the model.

For Iris, the 4 raw features are numeric values, so we'll build a list of feature columns to tell the Estimator model to represent each of the four features as 32-bit floating-point values. Therefore, the code to create the feature column is:

In [6]:
import iris_data
import tensorflow as tf

# Fetch the data
(train_x, train_y), (test_x, test_y) = iris_data.load_data()
    
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

Feature columns can be far more sophisticated than those we're showing here. We detail feature columns [later on](https://tensorflow.google.cn/get_started/feature_columns) in our Getting Started guide.

Now that we have the description of how we want the model to represent the raw features, we can build the estimator.

## Instantiate an estimator
The Iris problem is a classic classification problem. Fortunately, TensorFlow provides several pre-made classifier Estimators, including:

- [tf.estimator.DNNClassifier](https://tensorflow.google.cn/api_docs/python/tf/estimator/DNNClassifier) for deep models that perform multi-class classification.
- [tf.estimator.DNNLinearCombinedClassifier](https://tensorflow.google.cn/api_docs/python/tf/estimator/DNNLinearCombinedClassifier) for wide & deep models.
- [tf.estimator.LinearClassifier](https://tensorflow.google.cn/api_docs/python/tf/estimator/LinearClassifier) for classifiers based on linear models.

For the Iris problem, [tf.estimator.DNNClassifier](https://tensorflow.google.cn/api_docs/python/tf/estimator/DNNClassifier) seems like the best choice. Here's how we instantiated this Estimator:

In [7]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.

classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ADMINI~1\\AppData\\Local\\Temp\\tmp9fisv8mz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022A4175A898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate, and Predict
Now that we have an Estimator object, we can call methods to do the following:

Train the model.
Evaluate the trained model.
Use the trained model to make predictions.
Train the model
Train the model by calling the Estimator's train method as follows:

In [10]:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, 100),
    steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMINI~1\AppData\Local\Temp\tmp9fisv8mz\model.ckpt.
INFO:tensorflow:loss = 108.55365, step = 1
INFO:tensorflow:global_step/sec: 506.838
INFO:tensorflow:loss = 12.782049, step = 101 (0.198 sec)
INFO:tensorflow:global_step/sec: 730.445
INFO:tensorflow:loss = 9.355957, step = 201 (0.137 sec)
INFO:tensorflow:global_step/sec: 694.873
INFO:tensorflow:loss = 6.908679, step = 301 (0.144 sec)
INFO:tensorflow:global_step/sec: 709.658
INFO:tensorflow:loss = 7.738742, step = 401 (0.142 sec)
INFO:tensorflow:global_step/sec: 685.354
INFO:tensorflow:loss = 8.495692, step = 501 (0.145 sec)
INFO:tensorflow:global_step/sec: 709.656
INFO:tensorflow:loss = 6.2040186, step = 601 (0.142 sec)
INFO:tensorflow:global_step/s

Here we wrap up our input_fn call in a [lambda](https://docs.python.org/3/tutorial/controlflow.html) to capture the arguments while providing an input function that takes no arguments, as expected by the Estimator. The steps argument tells the method to stop training after a number of training steps.

## Evaluate the trained model
Now that the model has been trained, we can get some statistics on its performance. The following code block evaluates the accuracy of the trained model on the test data:

In [13]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y, 100))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-06-18-15:02:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\tmp9fisv8mz\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-06-18-15:02:49
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.04775909, global_step = 1000, loss = 1.4327726

Test set accuracy: 0.967



Unlike our call to the train method, we did not pass the steps argument to evaluate. Our eval_input_fn only yields a single [epoch](https://developers.google.cn/machine-learning/glossary/#epoch) of data.

## Making predictions (inferring) from the trained model
We now have a trained model that produces good evaluation results. We can now use the trained model to predict the species of an Iris flower based on some unlabeled measurements. As with training and evaluation, we make predictions using a single function call:

In [37]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(features=predict_x, labels=None, batch_size=100))

The predict method returns a Python iterable, yielding a dictionary of prediction results for each example. The following code prints a few predictions and their probabilities:

In [38]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\tmp9fisv8mz\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 10.996462 ,   4.0547156, -18.235981 ], dtype=float32), 'probabilities': array([9.9903440e-01, 9.6564693e-04, 2.0141500e-13], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}

Prediction is "Setosa" (99.9%), expected "Setosa"
{'logits': array([-6.8780317,  1.6417279, -4.9893894], dtype=float32), 'probabilities': array([1.9918488e-04, 9.9848419e-01, 1.3166900e-03], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}

Prediction is "Versicolor" (99.8%), expected "Versicolor"
{'logits': array([-14.130454  ,  -2.5689547 ,   0.80324566], dtype=float32), 'probabilities': array([3.1602693e-07, 3

## Summary
Pre-made Estimators are an effective way to quickly create standard models.

Now that you've gotten started writing TensorFlow programs, consider the following material:

- [Checkpoints](https://tensorflow.google.cn/get_started/checkpoints) to learn how to save and restore models.
- [Datasets](https://tensorflow.google.cn/get_started/datasets_quickstart) to learn more about importing data into your model.
- [Creating Custom Estimators](https://tensorflow.google.cn/get_started/custom_estimators) to learn how to write your own Estimator, customized for a particular problem.